In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Suppose I have the population given by [0.2, 0.3, 0.5, 0]

In [64]:
pop_fraction= [0.5, 0.2, 0.3]

In [65]:
%run ../python/network.py
%run ../python/selfish_routing.py

In [66]:
G = create_braess_network()

In [67]:
test = Network(G, 3)

In [68]:
test.payoff

{(0, 0, 0, 3): array([ 0,  0,  0, 13]),
 (0, 0, 1, 2): array([ 0,  0,  4, 13]),
 (0, 0, 2, 1): array([ 0,  0,  5, 13]),
 (0, 0, 3, 0): array([0, 0, 6, 0]),
 (0, 1, 0, 2): array([ 0, 11,  0, 12]),
 (0, 1, 1, 1): array([ 0, 12,  4, 12]),
 (0, 1, 2, 0): array([ 0, 13,  5,  0]),
 (0, 2, 0, 1): array([ 0, 12,  0, 11]),
 (0, 2, 1, 0): array([ 0, 13,  4,  0]),
 (0, 3, 0, 0): array([ 0, 13,  0,  0]),
 (1, 0, 0, 2): array([20,  0,  0, 12]),
 (1, 0, 1, 1): array([20,  0,  3, 12]),
 (1, 0, 2, 0): array([20,  0,  4,  0]),
 (1, 1, 0, 1): array([20, 11,  0, 11]),
 (1, 1, 1, 0): array([20, 12,  3,  0]),
 (1, 2, 0, 0): array([20, 12,  0,  0]),
 (2, 0, 0, 1): array([20,  0,  0, 11]),
 (2, 0, 1, 0): array([20,  0,  2,  0]),
 (2, 1, 0, 0): array([20, 11,  0,  0]),
 (3, 0, 0, 0): array([20,  0,  0,  0])}

Monte Carlo method

In [95]:
def sample_group(population, group_size):
    #basic check
    assert np.sum(population) == 1.0,  "Group has to be smaller than pop"
    #create a copy to mess around
    #original_population = np.copy(np.array(population, dtype=int))
    #number_of_strategies = len(original_population)
    #nothing sampled at begining
    sample = np.zeros_like(population)
    for _ in range(group_size):
        # determine "winner"
        position = np.random.choice(range(len(population)), 1, p=population)[0] 
        # update sample and pop
        sample[position] = sample[position] +1 
        #original_population[position] = original_population[position] -1    
    return sample

In [96]:
from collections import Counter

In [97]:
pop = [0.5, 0.2, 0.3]
group_size = 3
number_of_samples = 500000
ans = []
for _ in range(number_of_samples):
    sample = tuple(sample_group(pop, group_size))
    ans.append(sample)
counter = Counter(ans)

In [98]:
for key in counter:
    print(key, end=" ")
    print(counter[key]/number_of_samples)

(0.0, 3.0, 0.0) 0.007886
(3.0, 0.0, 0.0) 0.124566
(1.0, 2.0, 0.0) 0.060552
(2.0, 1.0, 0.0) 0.149962
(2.0, 0.0, 1.0) 0.226004
(0.0, 1.0, 2.0) 0.05367
(1.0, 0.0, 2.0) 0.134956
(1.0, 1.0, 1.0) 0.179856
(0.0, 0.0, 3.0) 0.02654
(0.0, 2.0, 1.0) 0.036008


Multinomial Distribution 

In [99]:
def multiply_fictoral_of_values(x):
    to_return = 1
    for i in range(len(x)):
        to_return *= math.factorial(x[i])
    return to_return

In [100]:
def multiply_raise_to_power_of_values(propotion, raise_factor):
    to_return = 1
    for K_i, k_i in zip(propotion, raise_factor):
        to_return *= math.pow(K_i,k_i)
    return to_return

In [101]:
def multinomial_dist(pop_fractions, group_composition):
    t1=math.factorial(sum(group_composition))
    t2=multiply_fictoral_of_values(group_composition)
    t3=multiply_raise_to_power_of_values(pop_fractions, group_composition)
    t4 =((t1/t2)*t3)
    return t4
    

In [103]:
multinomial_dist(pop, [0, 3, 0])

0.008000000000000002

Calculationg payoffs

In [104]:
def calculating_payoffs():
    payoff = np.zeros_like(test.strategy_set)
    for i in test.payoff:
        payoff += multinomial_dist(pop_fraction, i) * test.payoff[i]
    return payoff

In [105]:
calculating_payoffs()

array([92.5, 24.764000000000003, 11.196, 84.7], dtype=object)

In [92]:
0.18119 + 0.007862 + 0.125708 + 0.150528 + 0.22459 + 0.053714 + 0.059998 + 0.133904 + 0.026886+ 0.03562

1.0

In [106]:
space = [(1.0, 1.0, 1.0),
(0.0, 3.0, 0.0),
(3.0, 0.0, 0.0),
(2.0, 1.0, 0.0), 
(2.0, 0.0, 1.0),
(0.0, 1.0, 2.0), 
(1.0, 2.0, 0.0), 
(1.0, 0.0, 2.0), 
(0.0, 0.0, 3.0), 
(0.0, 2.0, 1.0) ]

In [107]:
s = 0
for t in space:
    s += multinomial_dist([0.5, 0.2, 0.3], t)
s


1.0